# Explore the Caffe Model checkpoint!
Let's get the stuff converted!

In [1]:
import os
import math
import copy
import random
from namedlist import namedlist

import numpy as np
import tensorflow as tf
import cv2

slim = tf.contrib.slim

In [2]:
import sys
sys.path.append('../')

In [3]:
from pprint import pprint

import caffe
import numpy as np
from caffe.proto import caffe_pb2

In [4]:
caffe_filename = '../checkpoints/mobilenet.caffemodel'

caffemodel_params = caffe_pb2.NetParameter()
caffemodel_str = open(caffe_filename, 'rb').read()
caffemodel_params.ParseFromString(caffemodel_str)

In [5]:
layers = caffemodel_params.layer
names = [(i, l.name, l.type, l.blobs[0].shape.dim if len(l.blobs) else 0) for i, l in enumerate(layers)]
pprint(names)

[(0, 'data', 'ImageData', 0),
 (1, 'label_data_1_split', 'Split', 0),
 (2, 'conv1', 'Convolution', [32, 3, 3, 3]),
 (3, 'conv1/bn', 'BatchNorm', [32]),
 (4, 'conv1/scale', 'Scale', [32]),
 (5, 'relu1', 'ReLU', 0),
 (6, 'conv2_1/dw', 'Convolution', [32, 1, 3, 3]),
 (7, 'conv2_1/dw/bn', 'BatchNorm', [32]),
 (8, 'conv2_1/dw/scale', 'Scale', [32]),
 (9, 'relu2_1/dw', 'ReLU', 0),
 (10, 'conv2_1/sep', 'Convolution', [64, 32, 1, 1]),
 (11, 'conv2_1/sep/bn', 'BatchNorm', [64]),
 (12, 'conv2_1/sep/scale', 'Scale', [64]),
 (13, 'relu2_1/sep', 'ReLU', 0),
 (14, 'conv2_2/dw', 'Convolution', [64, 1, 3, 3]),
 (15, 'conv2_2/dw/bn', 'BatchNorm', [64]),
 (16, 'conv2_2/dw/scale', 'Scale', [64]),
 (17, 'relu2_2/dw', 'ReLU', 0),
 (18, 'conv2_2/sep', 'Convolution', [128, 64, 1, 1]),
 (19, 'conv2_2/sep/bn', 'BatchNorm', [128]),
 (20, 'conv2_2/sep/scale', 'Scale', [128]),
 (21, 'relu2_2/sep', 'ReLU', 0),
 (22, 'conv3_1/dw', 'Convolution', [128, 1, 3, 3]),
 (23, 'conv3_1/dw/bn', 'BatchNorm', [128]),
 (24, 'co

In [6]:
idx = 2
layer = layers[idx]
print(layer.type)
a = np.array(layer.blobs[0].data)
s = layer.blobs[0].shape.dim
print(s, 38*38)

Convolution
[32, 3, 3, 3] 1444


# Try to get it converted...


In [7]:
import tensorflow as tf

isess = tf.InteractiveSession()

In [8]:
from nets import mobilenets

In [9]:
# Load caffe model.
caffe_filename = '../checkpoints/mobilenet.caffemodel'
caffemodel_params = caffe_pb2.NetParameter() 
caffemodel_params.ParseFromString(open(caffe_filename, 'rb').read())

In [10]:
# MobileNets TF model
imgsize = mobilenets.mobilenets.default_image_size
shape = (1, imgsize, imgsize, 3)
img_input = tf.placeholder(shape=shape, dtype=tf.float32)
# Create model.
with slim.arg_scope(mobilenets.mobilenets_arg_scope(data_format='NHWC')):
    out = mobilenets.mobilenets(img_input, num_classes=1000, is_training=False)

# TF stuff!
tf.logging.set_verbosity(tf.logging.INFO)
global_step = slim.create_global_step()



TypeError: separable_convolution2d() got an unexpected keyword argument 'data_format'

In [ ]:
caffemodel = caffe_scope.CaffeScope()
caffemodel.load(FLAGS.caffemodel_path)

tf.logging.set_verbosity(tf.logging.INFO)
with tf.Graph().as_default():
    global_step = slim.create_global_step()
    num_classes = int(FLAGS.num_classes)

    # Select the network.
    ssd_class = nets_factory.get_network(FLAGS.model_name)
    ssd_params = ssd_class.default_params._replace(num_classes=num_classes)
    ssd_net = ssd_class(ssd_params)
    ssd_shape = ssd_net.params.img_shape

    # Image placeholder and model.
    shape = (1, ssd_shape[0], ssd_shape[1], 3)
    img_input = tf.placeholder(shape=shape, dtype=tf.float32)
    # Create model.
    with slim.arg_scope(ssd_net.arg_scope_caffe(caffemodel)):
        ssd_net.net(img_input, is_training=False)

    init_op = tf.global_variables_initializer()
    with tf.Session() as session:
        # Run the init operation.
        session.run(init_op)

        # Save model in checkpoint.
        saver = tf.train.Saver()
        ckpt_path = FLAGS.caffemodel_path.replace('.caffemodel', '.ckpt')
        saver.save(session, ckpt_path, write_meta_graph=False)


In [11]:
tf.__version__

'1.1.0'